In [1]:
from pathlib import Path

import click
import pandas as pd
from recsys.config import Config

import recsys.algorithms.last_hotel_algorithm.rec_last_hotel as rec
import recsys.algorithms.utils as utils

In [2]:
config = Config()

# calculate path to files
default_data_directory = config.data_path
data_directory = default_data_directory
test_csv = data_directory.joinpath('test.csv')
subm_csv = data_directory.joinpath('submission_last_hotel.csv')

df_test = pd.read_csv(test_csv)

df_target = utils.get_submission_target(df_test)

In [132]:
def most_element(l):
    if len(l) != 0:
        return l[-1]
    else:
        return -1

In [7]:
def most_frequent(l): 
    
    if len(l) != 0:
        counter = 0
        num = l[0] 

        for i in l: 
            curr_frequency = l.count(i) 
            if(curr_frequency> counter): 
                counter = curr_frequency 
                num = i 
        return num
    else:
        return -1

In [8]:
df_grouped = df_test.groupby(['user_id', 'session_id'], as_index=False)['reference'].apply(list)
df_references = df_grouped.reset_index(name='references')

In [9]:
df_references['most_ref_hotel'] = df_references['references'].apply(lambda x: most_frequent([e for e in x if str(e).isdigit()]))

In [10]:
df_last_ref = df_references[df_references['most_ref_hotel'] != -1][['user_id', 'session_id', 'most_ref_hotel']]

In [11]:
df_target['impressions'] = df_target['impressions'].apply(utils.string_to_array)

/data/recsys-challenge-2019/recsys_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df_joined = df_target.merge(df_last_ref, on=['user_id', 'session_id'], how='left')

In [13]:
def reorder(x):
    impressions = x['impressions'].copy()
    last_ref_hotel = str(x['most_ref_hotel'])
    if last_ref_hotel in impressions:
        impressions.remove(last_ref_hotel)
        impressions = [last_ref_hotel] + impressions
    return impressions

In [14]:
df_joined['impressions_reordered'] = df_joined.apply(reorder, axis=1)

In [15]:
df_joined['impressions_reordered'] = df_joined['impressions_reordered'].apply(lambda x: ' '.join(x))